In [39]:
!pip install tensorflow  opencv-python mediapipe sklearn matplotlib pillow

In [40]:
pip install pysocks scikit-learn  

Note: you may need to restart the kernel to use updated packages.


In [120]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont
import datetime

In [121]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [122]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 
current_directory = os.getcwd()
# Actions that we try to detect
#actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ','ඔ','ඕ','ක්','ග්','ජ්','ට්','ද්','ණ්','ත්','ඩ්','න්','ප්','බ්','ම්','ය්','ර්','ල්','ව්','ස්','හ්'])
# length_of_array = len(actions)
# print(length_of_array)
actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ','ඊ','උ','ඌ','එ','ඒ'])	
# actions = np.array(['අ', 'ආ', 'ඇ','ඈ','ඉ'])	
# actions = np.array(['ඊ','උ','ඌ','එ','ඒ'])
# Thirty videos worth of data
no_sequences = 30

# Each One Video  going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 0

In [123]:
label_map = {label:num for num, label in enumerate(actions)}

In [125]:
sequences, labels = [], []
# loop each action
for action in actions:
    # Loop through the each action sequence(videos)
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        # this array will hold all the frame in one video
        window = []
        # Loop though the each frame 30(Frame)
        for frame_num in range(sequence_length):
            file_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))

            if os.path.exists(file_path):
                res = np.load(file_path)
            else:
                print(f"Error: File not found at {file_path}")
            # # using np.load function load the saved np array from folder (path,which action,which video number,which frame number)
            # res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            # add loaded framee to window
            window.append(res)
        #    
        sequences.append(window)
        labels.append(label_map[action])

In [126]:
# shape should be 900(30 action 1 action has 30 videos) videos,30 frame each,126(21*3*2) key points in a frame
np.array(sequences).shape

(300, 30, 126)

In [127]:
np.array(labels).shape

(300,)

In [128]:
X = np.array(sequences)

In [129]:
y = to_categorical(labels).astype(int)

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [131]:
from tensorflow.keras.models import Sequential
# Make us able to action detection
from tensorflow.keras.layers import LSTM, Dense
# Make us able to monitor our modle during taining
from tensorflow.keras.callbacks import TensorBoard

In [132]:
# Get the current date in YYYYMMDD format
current_date_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint_path = "checkpoints/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Check if the checkpoint directory exists, if not, create it
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

# Check if a latest checkpoint exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,  # Path to save the model weights
    save_weights_only=True,     # Save only the model weights
    verbose=1,                  # Verbosity mode, 1 for progress bar
    save_freq=50          
)



In [133]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [134]:
# load Sequential api
model = Sequential()
# add 3 layers of lstm model
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [135]:
# If a latest checkpoint exists, load the weights and continue training
# if latest_checkpoint:
#     # print(f"Loading weights from checkpoint: {latest_checkpoint}")
#     model.load_weights(latest_checkpoint)
# else:
#     if os.path.exists('action.h5'):
#         # Load the pre-trained model from the H5 file
#         model = tf.keras.models.load_model('action.h5')

#         # Get the model summary
#         model.summary()

        # # Get the model weights
        # model_weights = loaded_model.get_weights()
        # model = model.set_weights(model_weights)



In [136]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f"Test Accuracy: {accuracy:.2f}")

1/1 [==============================] - 1s 763ms/step - loss: 2.3141 - categorical_accuracy: 0.0000e+00
Test Accuracy: 0.00


In [137]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback,cp_callback])

Epoch 1/2000
9/9 [==============================] - 5s 34ms/step - loss: 2.2981 - categorical_accuracy: 0.1053
Epoch 2/2000
9/9 [==============================] - 0s 33ms/step - loss: 2.2000 - categorical_accuracy: 0.1053
Epoch 3/2000
9/9 [==============================] - 0s 33ms/step - loss: 2.1607 - categorical_accuracy: 0.0947
Epoch 4/2000
9/9 [==============================] - 0s 34ms/step - loss: 2.1267 - categorical_accuracy: 0.1509
Epoch 5/2000
9/9 [==============================] - 0s 33ms/step - loss: 2.0380 - categorical_accuracy: 0.1614
Epoch 6/2000
3/9 [=========>....................] - ETA: 0s - loss: 1.9405 - categorical_accuracy: 0.2188
Epoch 6: saving model to checkpoints\cp-0006.ckpt
9/9 [==============================] - 0s 40ms/step - loss: 1.9913 - categorical_accuracy: 0.2070
Epoch 7/2000
9/9 [==============================] - 0s 34ms/step - loss: 1.8884 - categorical_accuracy: 0.2456
Epoch 8/2000
9/9 [==============================] - 0s 33ms/step - loss: 1.7575 

KeyboardInterrupt: 

In [138]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 30, 64)            48896     
                                                                 
 lstm_19 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_20 (LSTM)              (None, 64)                49408     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 10)                330       
                                                                 
Total params: 203690 (795.66 KB)
Trainable params: 203

In [139]:
res = model.predict(X_test)

1/1 [==============================] - 0s 366ms/step


In [140]:
print(res)

[[2.40599945e-07 3.18861777e-12 1.89500264e-04 1.08517917e-04
  9.48815401e-11 1.25660961e-06 1.77616093e-04 9.80501136e-05
  6.33874379e-06 9.99418497e-01]
 [2.09859809e-06 3.12927934e-10 2.84362137e-01 7.13697672e-01
  4.65162975e-06 9.51312245e-07 6.35822425e-06 8.96655001e-06
  1.09118787e-07 1.91711658e-03]
 [9.99995470e-01 4.01217994e-06 8.86338691e-13 2.54601488e-19
  1.45345671e-11 2.29975989e-23 5.50536105e-22 1.35536716e-23
  4.25923474e-07 3.75922750e-24]
 [1.19901251e-03 4.35315887e-04 2.24113790e-03 5.14956890e-03
  9.90937412e-01 1.79242443e-06 1.05608831e-06 1.36896460e-07
  3.33286043e-05 1.26524333e-06]
 [4.92708106e-03 4.84178745e-04 1.35811567e-02 2.85145342e-02
  9.52297330e-01 2.39859060e-06 1.02192007e-06 1.24352354e-07
  1.87612633e-04 4.52473614e-06]
 [1.93845562e-09 3.04515290e-14 3.86182683e-06 2.18488458e-06
  5.34224141e-13 5.07588538e-07 2.56644911e-04 4.11869223e-05
  3.94937302e-07 9.99695301e-01]
 [1.62795714e-05 1.36792631e-08 2.41361201e-01 7.57923245e

In [141]:
actions[np.argmax(res[4])]

'ඉ'

In [142]:
actions[np.argmax(y_test[4])]

'ඉ'

In [143]:
model.save('action.keras')